In [1]:
# import the necessary libraries
import os
import nltk
import string
import re
import sys
import json
from nltk.corpus import stopwords
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
nltk.download('punkt')
nltk.download('stopwords')
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Priyanshu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Priyanshu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from preprocess import preprocess


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Priyanshu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Priyanshu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Global Variables

In [3]:
len_docs = []
individual_tokens = []

Positional Index Generation

In [4]:
path = r'C:\Users\Priyanshu\Downloads\IR Ass2\CSE508_Winter2023_Dataset'

pos_index_dict = {}
file_id_map_dict={}
file_index = 1
unprocessed_filelist = []


for filename in os.listdir(path):
  
  pos = 0

  file_id_map_dict[filename]=file_index

  try:
    fp=open(os.path.join(path, filename),'r')

    # tokens,preprocessed_text = preprocess(fp.read)
    tokens=fp.read().split(" ")
    individual_tokens.append(tokens)
    len_docs.append(len(tokens))

    for token in tokens:

      if token in pos_index_dict:

        pos_index_dict[token][0] += 1
        
        if file_index in pos_index_dict[token][1]:
          pos_index_dict[token][2][file_index].append(pos)
        else:
          pos_index_dict[token][1].append(file_index)
          pos_index_dict[token][2][file_index] = [pos]
      
      else:
        pos_index_dict[token] = [1,[file_index],{file_index:[pos]}]
      pos += 1

  except:
      unprocessed_filelist.append(file_index)

  file_index += 1

  with open('pos_index_dict','wb') as fd:
    pickle.dump(pos_index_dict,fd)
  with open('file_id_map_dict','wb') as fd:
    pickle.dump(file_id_map_dict,fd)
  with open('individual_tokens','wb') as fd:
    pickle.dump(individual_tokens,fd)




In [ ]:
# with open('pos_index_dict','rb') as fd:
#     pos_index_dict=pickle.load(fd)
# with open('file_id_map_dict','rb') as fd:
#     file_id_map_dict=pickle.load(fd)
# with open('individual_tokens','rb') as fd:
#     individual_tokens=pickle.load(fd)

inv_file_id_map_dict={v: k for k, v in file_id_map_dict.items()}

TF-IDF Matrix Generation

In [5]:
num_documents = len(file_id_map_dict)
vocab_size = len(pos_index_dict)
vocab = list(pos_index_dict.keys())

In [6]:
def createMatrix(scheme):
    if scheme=='binary':
       from scheme import get_tf_binary as get_tf
    elif scheme=='raw_count':
       from scheme import get_tf_raw_count as get_tf
    elif scheme=='term_frequency':
       from scheme import get_tf_term_frequency as get_tf
    elif scheme=='log_normalization':
       from scheme import get_tf_logNorm as get_tf
    else:
       from scheme import get_tf_logDoubleNorm as get_tf
        
    matrix = np.empty((num_documents,vocab_size))

    for i in range(num_documents):
        for j in range(vocab_size):

            curr_word = vocab[j]
            doc_list = pos_index_dict[curr_word][1]
            tf = 0

            if i+1 in doc_list:
                tf=get_tf(curr_word,i+1,pos_index_dict,len_docs,individual_tokens)

            idf = math.log((num_documents*1.0)/(len(doc_list)+1),2)

            matrix[i][j] = tf*idf 
    return matrix

In [7]:
# matrix1 = createMatrix('binary')

In [8]:
# matrix2 = createMatrix('raw_count')

In [9]:
# matrix3 = createMatrix('term_frequency')

In [10]:
# matrix4 = createMatrix('log_normalization')

In [11]:
matrix5 = createMatrix('double_normalization')
with open('matrix5','wb') as fd:
    pickle.dump(matrix5,fd)


In [ ]:
# with open('matrix5','rb') as fd:
#     matrix5=pickle.load(fd)

TF-IDF

In [24]:
print("Input the query")
query = str(input())
print("Yourr query is : "+query)


tokens,preprocessed_text = preprocess(query)
print(" Preprocessed query : "+ str(tokens))
indices = []
doc_score = {}


for t in tokens:
    try:
        indices.append(vocab.index(t))
    except:
        print("Unmatched Token Error : "+str(t))

for i in range(num_documents):
  score = 0

  for j in indices:
    score = score + matrix5[i][j]
  
  doc_score[i+1] = score
  
marklist=reversed(sorted(doc_score.items(), key=lambda x:x[1]))
sortdict = dict(marklist)

answer = list(sortdict.keys())
for i in range(5):
  print(inv_file_id_map_dict[answer[i]], sortdict[answer[i]])


Input the query
Yourr query is : removing finding doing stuff and forgetting who peoples are \t
 Preprocessed query : ['removing', 'finding', 'stuff', 'forgetting', 'peoples']
Unmatched Token Error : stuff
Unmatched Token Error : forgetting
Unmatched Token Error : peoples
cranfield0738 7.129283016944966
cranfield1323 5.910832407407448
cranfield0745 5.3469622627087245
cranfield1205 5.171978356481518
cranfield0556 4.752855344629977


Jaccard Coefficient

In [34]:
print("Input the phrase query")
query = str(input())
print(" Your query : "+ str(query))
tokens, preprocessed_text = preprocess(query)
query = set(tokens)

print(" Preprocessed query : "+ str(tokens))

doc_score = {}

for i in range(len(file_id_map_dict.keys())):

  doc = set(individual_tokens[i])
  num = len(list(doc & query))*1.0
  den = len(list(doc | query))
  doc_score[i] = num/den

marklist = sorted(doc_score.items(), key=lambda x:x[1])
sortdict = dict(reversed(marklist))

answer = list(sortdict.keys())
for i in range(5):
  print(inv_file_id_map_dict[int(answer[i])+1], sortdict[answer[i]])

Input the phrase query
 Your query : solutions thethree particular casesi propose give general solutionto problem indicatebriefly obtained onedimensional transient heat temperature doublelayer slab triangular heatrate input one face insulated otherand thermal resistanceat interface flow multilayerslab recent contribution readersforum wassermann gave analyticsolutions 
 Preprocessed query : ['solutions', 'thethree', 'particular', 'casesi', 'propose', 'give', 'general', 'solutionto', 'problem', 'indicatebriefly', 'obtained', 'onedimensional', 'transient', 'heat', 'temperature', 'doublelayer', 'slab', 'triangular', 'heatrate', 'input', 'one', 'face', 'insulated', 'otherand', 'thermal', 'resistanceat', 'interface', 'flow', 'multilayerslab', 'recent', 'contribution', 'readersforum', 'wassermann', 'gave', 'analyticsolutions']
cranfield0006 0.7446808510638298
cranfield0091 0.13333333333333333
cranfield0582 0.11594202898550725
cranfield0005 0.10526315789473684
cranfield0981 0.08536585365853659